<a href="https://colab.research.google.com/github/juooo1117/cyber_security_project/blob/main/cyber_security_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 로드**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import csv
import numpy as np

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score

# **데이터 전처리**

In [4]:
con1 = pd.read_csv('/content/drive/MyDrive/A_Track_trainset.csv')
con2 = pd.read_csv('/content/drive/MyDrive/A_Track_trainset.csv')

df = pd.concat([con1,con2])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90000 entries, 0 to 44999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Log_Number    90000 non-null  int64 
 1   payload       90000 non-null  object
 2   label_action  90000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [6]:
df.head(20)

,Log_Number,payload,label_action
0,0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
5,5,POST /member/login HTTP/1.1\r\nContent-Length:...,Vulnerability_Scan
6,6,GET /board/?category=&gdviewer=1&goods_seq=69&...,System_Cmd_Execution
7,7,GET /board/board_list?code=welfare&search_type...,SQL_Injection
8,8,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,HOST_Scan
9,9,GET / HTTP/1.1\r\n\r\n,HOST_Scan


In [7]:
df.columns

Index(['Log_Number', 'payload', 'label_action'], dtype='object')

In [8]:
# label 종류('label_action') -> 총 9개
df['label_action'].unique()

array(['System_Cmd_Execution', 'SQL_Injection', 'HOST_Scan',
       'Vulnerability_Scan', 'Path_Disclosure', 'Cross_Site_Scripting',
       'Leakage_Through_NW', 'Directory_Indexing',
       'Automatically_Searching_Infor'], dtype=object)

In [9]:
# label category의 각각 값 개수
df['label_action'].value_counts()

Vulnerability_Scan               33734
System_Cmd_Execution             19614
HOST_Scan                        12630
Path_Disclosure                   9550
SQL_Injection                     6790
Cross_Site_Scripting              2696
Automatically_Searching_Infor     2238
Leakage_Through_NW                1912
Directory_Indexing                 836
Name: label_action, dtype: int64

In [10]:
# Log_Number column 삭제
df = df.drop('Log_Number', axis = 1)
df.head(20)

,payload,label_action
0,"GET /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,POST /owa/auth/logon.aspx?replaceCurrent=1%22%...,System_Cmd_Execution
2,GET /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,GET / HTTP/1.1\r\n\r\n,HOST_Scan
4,GET /sub_04_1_read.php?page=1&id=31%29%3BSELEC...,System_Cmd_Execution
5,POST /member/login HTTP/1.1\r\nContent-Length:...,Vulnerability_Scan
6,GET /board/?category=&gdviewer=1&goods_seq=69&...,System_Cmd_Execution
7,GET /board/board_list?code=welfare&search_type...,SQL_Injection
8,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,HOST_Scan
9,GET / HTTP/1.1\r\n\r\n,HOST_Scan


In [11]:
# 중복값제거
# payload 가 같으나 label_action 이 다른 데이터 -> 해당 payload의 label_action별 카운트 중 빈도가 높은 label_action으로 변경
#                                                                        카운트가 같다면 전체 label_action 별 카운트 빈도가 높은 label_action으로 변경

drop_df = df.drop_duplicates()
x = drop_df[drop_df.duplicated(['payload'], keep=False)].drop_duplicates(subset = ['payload'])['payload']

for i in x:

    v = df[df['payload'] == i]['label_action'].mode().values

    if len(v) == 1:
        df.loc[(df['payload'] == i), 'label_action'] = v[0]

    else:
        fvc = 0
        for j in df[df['payload'] == i]['label_action'].unique():
            vc = df['label_action'].value_counts().loc[j]
            if vc > fvc :
                fvc = vc
                m = j
            else :
                pass
        df.loc[(df['payload'] == i), 'label_action'] = m

In [12]:
# 각 label_action 값 변화된 것 확인 -> 총 개수는 동일(45000개)
df['label_action'].value_counts()

Vulnerability_Scan               33764
System_Cmd_Execution             19594
HOST_Scan                        12630
Path_Disclosure                   9542
SQL_Injection                     6788
Cross_Site_Scripting              2696
Automatically_Searching_Infor     2238
Leakage_Through_NW                1912
Directory_Indexing                 836
Name: label_action, dtype: int64

In [13]:
# label_action: Vulnerability_Scan 인 값들 (취약한 값들 & 제일 값 많음 -> 맨 나중에??)
# 나머지 label_action category 별로도 payload 값을 확인했으나, 기준으로 삼을만한 패턴이 보이지 않음

df.loc[df['label_action'] == 'Vulnerability_Scan', ['label_action','payload']]

,label_action,payload
5,Vulnerability_Scan,POST /member/login HTTP/1.1\r\nContent-Length:...
10,Vulnerability_Scan,GET /mod/liens/index.php?config[pathMod]=http:...
16,Vulnerability_Scan,GET /postnuke/html/index.php?name=forums&file=...
19,Vulnerability_Scan,GET /goods/goods_search?display_type=list&arr_...
28,Vulnerability_Scan,GET /library/translation.inc.php?GLOBALS[srcdi...
...,...,...
44984,Vulnerability_Scan,GET /admin/login.php HTTP/1.0\r\nHost: 10.0.17...
44986,Vulnerability_Scan,GET /goods/goods_search?display_type=list&arr_...
44990,Vulnerability_Scan,GET /goods/goods_search?display_type=list&arr_...
44994,Vulnerability_Scan,POST /controller/board.php HTTP/1.1\r\nHost: l...


# **Vocab dictionary 만들기**

In [14]:
!pip3 install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.6 MB/s eta 0:00:00


In [15]:
import sentencepiece as spm

In [16]:
# payload 데이터만 .txt 파일로 저장
with open(r'/content/drive/MyDrive/payload.txt', 'w', encoding = 'utf-8') as f:
    for line in df.payload.values:
        try:
            f.write(line+'\n')
        except TypeError as TE:
            print(line, TE)

In [17]:
# payload 전체가 .txt 파일로 저장된 것 확인
with open(r'/content/drive/MyDrive/payload.txt', 'r', encoding = 'utf-8') as f:
    nsmc_text = f.read().split('\n')
print(nsmc_text[0])

GET /forum1_professionnel.asp?n=/.\\\"./.\\\"./.\\\"./.\\\"./.\\\"./boot.ini&amp;nn=100&amp;page=1|234|800a0bcd|Either_BOF_or_EOF_is_True__or_the_current_record_has_been_deleted._Requested_operation_requires_a_current_record. HTTP/1.1\r\nConnection: Keep-Alive\r\nUser-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None) (Test:002421)\r\nContent-Type: application/x-www-form-urlencoded\r\nContent-Length: 36\r\nHost: 10.0.17.21\r\n\r\n<!--#include virtual=\"/indomainabc063.jsp\"-->


In [18]:
# google sentencepiece 사용하기(토큰화)
input_file = r'/content/drive/MyDrive/payload.txt'

prefix = 'payload'
vocab_size = 32000
model_type = 'bpe'
character_coverage  = 1.0
max_sentence_length = 9999

In [19]:
templates = '--input={} --model_prefix={} --vocab_size={} --model_type={} --character_coverage={} --max_sentence_length={}'
cmd = templates.format(input_file, prefix, vocab_size, model_type, character_coverage, max_sentence_length)
cmd

'--input=/content/drive/MyDrive/payload.txt --model_prefix=payload --vocab_size=32000 --model_type=bpe --character_coverage=1.0 --max_sentence_length=9999'

In [20]:
# SentencePiece를 설정한 값들로(cmd) 훈련시켜서 단어집합 생성
spm.SentencePieceTrainer.Train(cmd)

In [21]:
# vocab 에서 학습된 subwords를 확인
# type(vocab_list) : dataframe

vocab_list = pd.read_csv('payload.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)

In [22]:
# vocab에 저장되어 있는 단어는 총 32000개(설정한 값)
len(vocab_list)

32000

In [ ]:
# 전체 payload를 32000개로 잘라서 각각의 단어에 고유한 번호를 부여
vocab_list

In [ ]:
# 훈련된 SentencePiece를(payload.model) model로 저장함
sp = spm.SentencePieceProcessor()
vocab_file = "payload.model"
sp.load(vocab_file)

In [25]:
# 저장된 모델을 사용했을때, SentencePiece가 잘 적용되는지 확인해보자
lines = ["GET /forum1_professionnel.asp?n=/.\\\"./.\\\"./.\\\"./.\\\"./.\\\"./boot.ini&amp;nn=100&amp;page=1|234|800a0bcd|Either_BOF_or_EOF_is_True__or_the_current_record_has_been_deleted._Requested_operation_requires_a_current_record. HTTP/1.1\r\nConnection: Keep-Alive\r\nUser-Agent: Mozilla/5.00 (Nikto/2.1.6) (Evasions:None) (Test:002421)\r\nContent-Type: application/x-www-form-urlencoded\r\nContent-Length: 36\r\nHost: 10.0.17.21\r\n\r\n<!--#include virtual=\"/indomainabc063.jsp\"-->",
         "POST /owa/auth/logon.aspx?replaceCurrent=1%22%29%20AND%209294%3DUTL_INADDR.GET_HOST_ADDRESS%28CHR%28113%29%7C%7CCHR%28112%29%7C%7CCHR%2898%29%7C%7CCHR%28120%29%7C%7CCHR%28113%29%7C%7C%28SELECT%20%28CASE%20WHEN%20%289294%3D9294%29%20THEN%201%20ELSE%200%20END%29%20FROM%20DUAL%29%7C%7CCHR%28113%29%7C%7CCHR%28122%29%7C%7CCHR%28106%29%7C%7CCHR%28107%29%7C%7CCHR%28113%29%29%20AND%20%28%22tdDX%22%3D%22tdDX&reason=3&url= HTTP/1.1\r\nContent-Length: 0\r\nCache-Control: no-cache\r\nUser-Agent: sqlmap/1.6.7#stable (https://sqlmap.org)\r\nReferer: http://owa.college.school:80/owa/auth/logon.aspx\r\nHost: owa.college.school\r\nAccept: */*\r\nAccept-Encoding: gzip,deflate\r\nContent-Type: application/x-www-form-urlencoded; charset=utf-8\r\nConnection: close\r\n\r\n",
         "GET /goods/goods_search?display_type=list&arr_search_list%5B0%5D%5Bsearch_type%5D=-1839%20or%201%3D2&arr_search_list%5B0%5D%5Bsearch%5D=1&arr_search_list%5B1%5D%5Bsearch_type%5D=Gm.name&arr_search_list%5B1%5D%5Bsearch%5D=1&arr_search_list%5B2%5D%5Bsearch_type%5D=Gm.name&arr_search_list%5B2%5D%5Bsearch%5D=1&search_include=y&search=1&search_type=Gm.name HTTP/1.1\r\nHost: www.college.school\r\nAuthorization: Negotiate TlRMTVNTUAABAAAAt4II4gAAAAAAAAAAAAAAAAAAAAAKAF1YAAAADw==\r\nAccept-Encoding: gzip, deflate\r\nUser-Agent: Mozilla/5.0 (Gecko) Arachni/v10.0.0.80\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8\r\nAccept-Language: en-US,en;q=0.8,he;q=0.6\r\nX-Arachni-Scan-Seed: 4818b1cc56353e785f9bfe9b055b7183\r\nCookie: designart_site=cg0q4hq5j1rgo9o2ns7s3nsnjlauq1rp\r\n\r\n"
         ]

# sp.encode_as_pieces() -> 각 payload 잘 토큰화된 것 확인!
for line in lines:
    #print(line)
    print(sp.encode_as_pieces(line))

['▁GET', '▁/', 'forum', '1_', 'professionnel', '.', 'asp', '?', 'n', '=/.', '\\".', '/.', '\\".', '/.', '\\".', '/.', '\\".', '/.', '\\"', './', 'boot', '.', 'ini', '&', 'amp', ';', 'nn', '=100&', 'amp', ';', 'page', '=1|234|800', 'a', '0', 'bcd', '|', 'Either', '_', 'BOF', '_', 'or', '_', 'EOF', '_', 'is', '_', 'True', '__', 'or', '_', 'the', '_', 'current', '_', 'record', '_', 'has', '_', 'been', '_', 'deleted', '._', 'Requested', '_', 'operation', '_', 'requires', '_', 'a', '_', 'current', '_', 'record', '.', '▁HTTP', '/1', '.1', '▁Con', 'nection', ':', '▁Keep', '-', 'Alive', '▁U', 'ser', '-', 'Agent', ':', '▁Mozilla', '/5.00', '▁(', 'Nikto', '/2.1.6)', '▁(', 'Evasions', ':', 'None', ')', '▁(', 'Test', ':00242', '1)', '▁Content', '-', 'Type', ':', '▁application', '/', 'x', '-', 'www', '-', 'form', '-', 'urlencoded', '▁Content', '-', 'Length', ':', '▁36', '▁H', 'ost', ':', '▁10.0.17.21', '▁', '<!--#', 'include', '▁virtual', '=', '"', '/', 'indomainabc', '063.', 'jsp', '"', '-->']
['▁

# **Vocab dictionary 를 적용**

In [26]:
token = []

# df 한줄씩 돌면서 df[payload]를 학습된 sentencepiece 모델을 적용해서 토큰화시킴 -> df[payload_token]을 새로 만들어서 이 토큰들을 저장!
for i in range(len(df)):
  if(i % 10000 == 0):
    print(i, "단계 완료")
  token.append(' '.join(sp.encode_as_pieces(str(df.iloc[i]['payload']))))  # 토큰들은 각각 공백을 기준으로 나열됨

df['payload_token'] = token

0 단계 완료
10000 단계 완료
20000 단계 완료
30000 단계 완료
40000 단계 완료
50000 단계 완료
60000 단계 완료
70000 단계 완료
80000 단계 완료


In [ ]:
# df[payload_token] column이 추가된 것 확인!
df.head(5)

In [28]:
# df['payload_token'] -> df_payload_token 으로 list (각 line당 공백으로 잘라서 list화)
# df['label_action'] -> df_label로 array

df_payload_token = list(df['payload_token'])
df_label = np.array(df['label_action'])

In [ ]:
# len(df_payload_token) -> 90000개
df_payload_token

In [59]:
# CountVectorizer 사용 -> 공백으로 나눔
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()

train_features=vectorizer.fit_transform(df_payload_token)

In [60]:
# 90000개의 payload 있는 것 확인!
print(train_features)

  (0, 9447)	1
  (0, 9094)	1
  (0, 2013)	1
  (0, 14142)	1
  (0, 6103)	1
  (0, 6639)	1
  (0, 10610)	1
  (0, 5945)	2
  (0, 13005)	1
  (0, 1631)	1
  (0, 13686)	1
  (0, 2628)	1
  (0, 4782)	1
  (0, 6324)	1
  (0, 8382)	1
  (0, 6624)	1
  (0, 13552)	2
  (0, 8498)	1
  (0, 10719)	1
  (0, 16229)	1
  (0, 5377)	1
  (0, 15985)	1
  (0, 7562)	2
  (0, 14713)	2
  (0, 9859)	1
  :	:
  (89999, 8454)	2
  (89999, 16653)	1
  (89999, 9923)	1
  (89999, 13239)	1
  (89999, 15171)	1
  (89999, 15233)	1
  (89999, 3749)	1
  (89999, 6963)	1
  (89999, 4025)	1
  (89999, 4739)	1
  (89999, 6411)	1
  (89999, 1497)	1
  (89999, 4531)	1
  (89999, 12749)	1
  (89999, 7796)	1
  (89999, 15406)	1
  (89999, 7081)	1
  (89999, 10153)	1
  (89999, 14829)	1
  (89999, 13134)	1
  (89999, 13153)	1
  (89999, 14991)	1
  (89999, 8907)	1
  (89999, 7614)	1
  (89999, 7615)	1


In [34]:
train_features.shape

(90000, 17903)

# **모델링 & 정확도확인**

In [35]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [36]:
# 훈련 데이터 셋과 검증 데이터 셋으로 분리

TEST_SIZE = 0.3
RANDOM_SEED = 1117

train_x, val_x, train_y, val_y = train_test_split(train_features, df_label, test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [37]:
# RandomForestClassifier (DecisionTreeClassifier는 정확도가 낮아서 사용X)
# n_estimators : boosting을 반복할 횟수(손실함수의 가중치를 몇 번이나 조정할 것인지)
#                값이 클수록 정확도가 증가하나, overfitting의 가능성 또한 증가

forest = RandomForestClassifier(n_estimators=100, random_state = RANDOM_SEED, class_weight = 'balanced')
forest.fit(train_x, train_y)

RandomForestClassifier(class_weight='balanced', random_state=1117)

In [38]:
# val data로 모델 accuracy 검증
forest.score(val_x, val_y)

0.9304444444444444

In [39]:
# .score(val_x, val_y) 라이브러리를 풀어쓴 코드 (동일한 accuracy 결과 출력!)
from sklearn.metrics import accuracy_score

predicted = forest.predict(val_x)
accuracy = accuracy_score(val_y, predicted)
print(accuracy)

0.9304444444444444


In [40]:
# crosstab: 각 'label_action' category별로 'pred' & 'real' 값을 교차 확인해서 피팅한 모델로 예측한 값이 얼마나 맞았는지 확인 (diagonal은 맞춘것!)
pred = forest.predict(val_x)
crosstab = pd.crosstab(val_y, pred, rownames=['real'], colnames=['pred'])
crosstab

pred,Automatically_Searching_Infor,Cross_Site_Scripting,Directory_Indexing,HOST_Scan,Leakage_Through_NW,Path_Disclosure,SQL_Injection,System_Cmd_Execution,Vulnerability_Scan
real,,,,,,,,,
Automatically_Searching_Infor,677,0,0,0,0,0,0,0,0
Cross_Site_Scripting,0,772,0,4,0,3,0,29,4
Directory_Indexing,0,0,240,0,0,0,0,0,0
HOST_Scan,0,4,2,3499,0,29,2,55,91
Leakage_Through_NW,0,0,0,0,572,0,0,0,0
Path_Disclosure,0,1,0,15,0,2699,0,100,99
SQL_Injection,0,0,0,0,0,0,1920,70,25
System_Cmd_Execution,0,69,3,40,0,124,41,5170,394
Vulnerability_Scan,4,7,3,145,2,130,18,365,9573


In [41]:
# 안전성 높이기 위해서 새로운 위험요소에 대한 가정을 추가하자
preds = forest.predict(val_x)
probas = forest.predict_proba(val_x)
print(preds.shape)
print(probas.shape)

(27000,)
(27000, 9)


In [42]:
# 예측치의 예측 확률이 0.7 이하인 경우(확신이 없을 경우) outlier로 판단
preds[np.where(np.max(probas, axis=1) < 0.7)] = 7
new_crosstab = pd.crosstab(val_y, preds, rownames=['real'], colnames=['pred'])
new_crosstab

pred,7,Automatically_Searching_Infor,Cross_Site_Scripting,Directory_Indexing,HOST_Scan,Leakage_Through_NW,Path_Disclosure,SQL_Injection,System_Cmd_Execution,Vulnerability_Scan
real,,,,,,,,,,
Automatically_Searching_Infor,4,673,0,0,0,0,0,0,0,0
Cross_Site_Scripting,92,0,698,0,0,0,0,0,20,2
Directory_Indexing,7,0,0,233,0,0,0,0,0,0
HOST_Scan,356,0,2,0,3285,0,6,0,21,12
Leakage_Through_NW,1,0,0,0,0,571,0,0,0,0
Path_Disclosure,340,0,0,0,7,0,2493,0,41,33
SQL_Injection,162,0,0,0,0,0,0,1827,20,6
System_Cmd_Execution,864,0,36,1,12,0,52,13,4689,174
Vulnerability_Scan,1242,4,5,0,36,0,30,10,129,8791


In [45]:
import pickle

#'forest_model'은 저장할 파일의 이름이다.
with open('forest_model', 'wb') as f:
    pickle.dump(forest, f)

# **test data 답 예측**

In [46]:
# 'forest_model'라는 이름에 저장한 모델 파일을 불러온다.
with open('forest_model', 'rb') as f:
    forest_model = pickle.load(f)

In [47]:
# 불러온 학습된 모델
forest_model

RandomForestClassifier(class_weight='balanced', random_state=1117)

In [48]:
test_df = pd.read_csv('/content/drive/MyDrive/A_Track_testset.csv')

In [49]:
# 모델에 넣어서 값 확인할 데이터 확인  (9977개)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9977 entries, 0 to 9976
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Log_Number  9977 non-null   int64 
 1   payload     9977 non-null   object
dtypes: int64(1), object(1)
memory usage: 156.0+ KB


In [50]:
test_df.head(10)

,Log_Number,payload
0,0,GET /jsp-examples/ HTTP/1.1\r\nContent-Length:...
1,1,POST /member/login.php HTTP/1.0\r\nHost: 10.0....
2,2,GET /phpBB/index.php?Nikto=Forums&file=viewtop...
3,3,GET /stats/search_auto_complete?keyword=%22%3E...
4,4,GET /board/?category=%3Bsleep+15%3B&goods_seq=...
5,5,GET /goods/goods_search?display_type=list&arr_...
6,6,GET /admin/login.php HTTP/1.0\r\nHost: 10.0.17...
7,7,POST /member/login HTTP/1.1\r\nContent-Length:...
8,8,20http://%s:%d/Mozi.m%20-O%20->%20/tmp/Netlink...
9,9,POST /controller/login.php HTTP/1.1\r\nContent...


In [51]:
# vocab dictionary로 tokenize
token = []

# 한줄씩 돌면서 test_df[payload]를 학습된 sentencepiece 모델을 적용해서 토큰화시킴 -> test_df[test_payload_token]을 새로 만들어서 이 토큰들을 저장!
for i in range(len(test_df)):
  if(i % 1000 == 0):
    print(i, "단계 완료")
  token.append(' '.join(sp.encode_as_pieces(str(test_df.iloc[i]['payload']))))  # 토큰들은 각각 공백을 기준으로 나열됨

test_df['test_payload_token'] = token

0 단계 완료
1000 단계 완료
2000 단계 완료
3000 단계 완료
4000 단계 완료
5000 단계 완료
6000 단계 완료
7000 단계 완료
8000 단계 완료
9000 단계 완료


In [52]:
test_df.head(10)

,Log_Number,payload,test_payload_token
0,0,GET /jsp-examples/ HTTP/1.1\r\nContent-Length:...,▁GET ▁/ jsp - examples / ▁HTTP /1.1\ r \ nCont...
1,1,POST /member/login.php HTTP/1.0\r\nHost: 10.0....,▁POST ▁/ member / login . php ▁HTTP /1.0\ r \ ...
2,2,GET /phpBB/index.php?Nikto=Forums&file=viewtop...,▁GET ▁/ phpBB / index . php ? Nikto = Forums &...
3,3,GET /stats/search_auto_complete?keyword=%22%3E...,▁GET ▁/ stats / search _ auto _ complete ? key...
4,4,GET /board/?category=%3Bsleep+15%3B&goods_seq=...,▁GET ▁/ board /? category =%3 Bsleep +15%3 B &...
5,5,GET /goods/goods_search?display_type=list&arr_...,▁GET ▁/ goods / goods _ search ? display _ typ...
6,6,GET /admin/login.php HTTP/1.0\r\nHost: 10.0.17...,▁GET ▁/ admin / login . php ▁HTTP /1.0\ r \ nH...
7,7,POST /member/login HTTP/1.1\r\nContent-Length:...,▁POST ▁/ member / login ▁HTTP /1.1\ r \ nConte...
8,8,20http://%s:%d/Mozi.m%20-O%20->%20/tmp/Netlink...,▁20 http ://% s :% d / Mozi . m %20- O %20->%2...
9,9,POST /controller/login.php HTTP/1.1\r\nContent...,▁POST ▁/ controller / login . php ▁HTTP /1.1\ ...


In [53]:
# test_df['test_payload_token'] => test_payload_token로 list화 (각 line당 공백으로 잘라서 list화)

test_payload_token = list(test_df['test_payload_token'])
test_lognumber = list(test_df['Log_Number'])

In [54]:
# 9977개 확인!
len(test_payload_token)

9977

In [61]:
# 'test_payload_token' list를 벡터화
test_features = vectorizer.transform(test_payload_token)

In [62]:
# 결과예측
results = forest_model.predict(test_features)
results_proba = forest.predict_proba(test_features)

In [64]:
print(results_proba)

[[0.         0.07       0.         ... 0.         0.         0.12      ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.00823961 0.99176039]
 ...
 [0.         0.         0.         ... 0.99       0.         0.01      ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.04       0.         ... 0.06       0.78399626 0.11600374]]


In [65]:
sub = pd.read_csv('/content/drive/MyDrive/A_Track_submission.csv')

In [66]:
sub.head(5)

,Log_Number,label_action
0,0,NaN
1,1,NaN
2,2,NaN
3,3,NaN
4,4,NaN


In [67]:
# submission 저장
sub['label_action']=results

In [68]:
sub['label_action'].value_counts()

Vulnerability_Scan               3569
System_Cmd_Execution             2226
HOST_Scan                        1440
Path_Disclosure                  1102
SQL_Injection                     791
Cross_Site_Scripting              299
Automatically_Searching_Infor     239
Leakage_Through_NW                209
Directory_Indexing                102
Name: label_action, dtype: int64

In [70]:
sub.to_csv('/content/drive/MyDrive/submission_result1.csv', index=False)

In [ ]:
# level이 2,4,6이 아니고(해당 category 값이 너무 적어서 outlier판단에서 제외) 예측치의 예측 확률이 0.7 이하, 즉 확신이 없을 경우 이상치로 판단
results_new = results.copy()
results_new[np.where((np.max(results_proba, axis=1) < 0.7) & (results != 6) & (results != 4) & (results != 2))] = 7